# EchoPro Kriging mesh Workflow <a class="tocSkip">

# Import libraries and configure the Jupyter notebook

In [ ]:
# libraries used in the Notebook
import matplotlib.pyplot as plt
import geopandas
import folium

# Python version of EchoPro
import EchoPro

# obtain all visualization routines
from EchoPro.export import plot_layered_points, plot_points

# Set up  EchoPro for a specific survey year

## Initialize EchoPro object  using configuration files

* `initialization_config.yml` -- parameters independent of survey year
* `survey_year_2019_config.yml` -- parameters specific to survey year
* `source` -- Define the region of data to use e.g. US, CAN, US & CAN
* `exclude_age1` -- States whether age 1 hake should be included in analysis.

In [ ]:
%%time 
survey_2019 = EchoPro.Survey(init_file_path='../config_files/initialization_config.yml',
                             survey_year_file_path='../config_files/survey_year_2019_config.yml',
                             source=3,
                             exclude_age1=True)

## Load and process input data 
* This data is stored in `survey_2019`

In [ ]:
%%time 
survey_2019.load_survey_data()

## Compute the areal biomass density
* The areal biomass density is stored in `survey_2019`

In [ ]:
%%time
survey_2019.compute_biomass_density()

# Obtain Kriging Mesh Data

## Access Kriging mesh object
* Reads mesh data files specified by `survey_2019` 

In [ ]:
krig_mesh = survey_2019.get_kriging_mesh()

### Plot the Mesh, Transects and smoothed isobath contour

* Generate interactive map using the Folium package
* Mesh points are in gray
* Transect points are represented by a changing color gradient
* Smoothed contour points (200m isobath) are in blue 

In [ ]:
fmap = plot_layered_points(krig_mesh)
fmap

## Obtain reduced mesh 
- EchoPro allows the user to select only those mesh points that are contained in a polygon surrounding the transect points
    - For more information on the method, please see the `Notes` section in the doc string for `KrigingMesh.get_polygon_of_transects`

In [ ]:
# obtain the polygon of the transects 
transect_polygon = krig_mesh.get_polygon_of_transects(survey_2019.bio_calc.final_biomass_table, 
                                                      n_close=4, nm_to_buffer=1.25)

In [ ]:
# plot transect points 
fmap_polygon = plot_points(survey_2019.bio_calc.final_biomass_table, cmap_column='transect_num', color='hex')

# Add polygon to folium map and display it
fmap_polygon.add_child(folium.GeoJson(transect_polygon))

### Select mesh points within polygon
- Using the constructed polygon we can get those mesh points that are within it

In [ ]:
# get reduced mesh based off of a polygon
reduced_mesh_gdf = krig_mesh.reduce_grid_points(transect_polygon)

In [ ]:
# plot the reduced mesh points
fmap_reduced = plot_points(reduced_mesh_gdf, color='red')

# plot the transect points 
fmap_reduced = plot_points(survey_2019.bio_calc.final_biomass_table, fmap_reduced, 
                           cmap_column='transect_num', color='hex')

# display transects and reduced mesh
fmap_reduced